# Importer les librairies, et fichiers .py auxiliaires

In [ ]:
import numpy as np
from preprocess_data import *
from bayes_classifier import BayesClassifier
from complement_naive_bayes import train_cnb_with_tfidf, train_cnb
from ensemble_learning import train_ensemble
from xgboost_classifier import train_xgboost, train_xgboost_with_tfidf
from svc_classifier import *
from sgd_classifier import *
from sklearn.model_selection import StratifiedKFold

from save_output import save_output

In [ ]:
data_preprocess = DataPreprocess()
X_train, y_train, X_test = data_preprocess.train, data_preprocess.label_train, data_preprocess.test

# Exploration de données

# Jalon 1) Naive de Bayes vanille
K-Fold Validation croisée ($k = 7)$:
- α: Lissage du postérieur de Bayes

In [ ]:
bayes_classifier = BayesClassifier()
best_lps, best_f1 = bayes_classifier.hyperparameter_tuning(X_train, y_train)
print(best_lps, best_f1)
tuned_bayes_classifier = BayesClassifier()
tuned_bayes_classifier.fit(X_train, y_train, best_lps)
predictions = np.array([tuned_bayes_classifier.predict(x_i) for x_i in X_test])
save_output(predictions, "bayes", best_lps, "stopwords")

# Jalon 2)
# Naive Bayes Complement
K-Fold validation croisée stratifiée ($k = 5)$:
- α: Lissage du postérieur de Bayes

Prétraitement: TFIDF

In [ ]:
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(X_train, y_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_50_iter", "tfidf")

Prétraitement: Sans prétraitement

In [ ]:
complement_naive_bayes = train_cnb(X_train, y_train)
predictions = complement_naive_bayes.predict(X_test)
save_output(predictions, "cnb", "random_search_50_iter", "no_preprocessing")

Prétraitement: TFIDF, réduction par arbre, stopwords retirés

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train, sorted_indeces_features = tree_based_dimensionality_reduction(data_preprocess.train, data_preprocess.label_train)
X_test = data_preprocess.test[:, sorted_indeces_features]
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(X_train, data_preprocess.label_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_50_iter", "tree_reduction_stopwords_tfidf")

Prétraitement: TFIDF, réduction par arbre, stopwords retirés, suréchantillonnage SMOTE

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train, sorted_indeces_features = tree_based_dimensionality_reduction(data_preprocess.train, data_preprocess.label_train)
X_test = data_preprocess.test[:, sorted_indeces_features]
X_train, y_train = smote_oversampling(data_preprocess.train, data_preprocess.label_train)

complement_naive_bayes = train_cnb(X_train, y_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)

save_output(predictions, "cnb", "random_search_50_iter", "tree_reduction_stopwords_tfidf_smote")

# SVC
Grid Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- $\gamma$ : Coefficient du Noyau RBF
- $C$ : Terme de régularization pour la pénalité euclidienne.

Prétraitement: Retrirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
indeces_to_remove = remove_cum_sum(data_preprocess.train, 0.95)
data_preprocess.train = np.delete(data_preprocess.train, indeces_to_remove, axis=1)
data_preprocess.test = np.delete(data_preprocess.test, indeces_to_remove, axis=1)
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

best_params_, best_score_ = train_svc(X_train_undersampled, y_train_undersampled)
print(best_params_, best_score_)
svc = SVC(kernel='rbf', C=best_params_['C'], gamma=best_params_['gamma'])
svc.fit(X_train_undersampled, y_train_undersampled)
y_pred = svc.predict(data_preprocess.test)
params = f"C={best_params_['C']}, gamma={best_params_['gamma']}"
save_output(y_pred, "svm", params, "stopwords_cum-sum_undersampled")

# SGDClassifier
Random Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- loss: ModifiedHuber 
- penalty: ElaticNet 
- l1_ratio: Porportion de la perte d'ElasticNet qui est l1.
- $\alpha$: Poids attribué au terme de régularisation. Une plus grande valeur favorise que certains coefficients soient annulés (par l1) ou fortement adoucis (par l2) quand la pénalité est ElaticNet. 

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
indeces_to_remove = remove_cum_sum(data_preprocess.train, 0.95)
data_preprocess.train = np.delete(data_preprocess.train, indeces_to_remove, axis=1)
data_preprocess.test = np.delete(data_preprocess.test, indeces_to_remove, axis=1)
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

best_sgd = train_ensemble(X_train_undersampled, y_train_undersampled)
predictions_voter = train_sgd.predict(data_preprocess.test)
save_output(predictions_voter, "sgd", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")

# XGBoost
Random Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- Learning rate: Taille du pas lors de la descente de critère sur la perte (gain de subdivision) 
- Nombre d'estimateurs (n_estimators): Nombre d'arbres de décision
- Profondeur maximale (max_depth): Profondeur maximale de chaque arbre (nombre de branchements max)
- sous-échantillon (subsample):  Proportion de données utilisées pour produire chaque arbre.

Prétraitement: Sur-échantillonnage bootstrap

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
x_train_oversampled, y_train_oversampled = boostrap_oversampling(data_preprocess.train, data_preprocess.label_train)
xgboost_classifier, tfidf_transformer = train_xgboost(x_train_oversampled, y_train_oversampled)
predictions = xgboost_classifier.predict(data_preprocess.test)
save_output(predictions, "xgboost", "random_search_15_iter", "stopwords_bootstrap")

# Apprentissage par Ensemble

# Ensemble: CNB, XGBoost, Logistic Regression

Prétraitement : sans stopwords, sous-échantillonage, réduction de dimension par somme cumulative

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
indeces_to_remove = remove_cum_sum(data_preprocess.train, 0.95)
data_preprocess.train = np.delete(data_preprocess.train, indeces_to_remove, axis=1)
data_preprocess.test = np.delete(data_preprocess.test, indeces_to_remove, axis=1)
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")

Prétraitement : sans stopwords, sous-échantillonage

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_15_iter", "stopwords_undersampling")

# Ensemble: Complement Naive Bayes, XGBoost, SVC, SGD

Prétraitement: sans stopwords, sous-échantillonage, réduction de dimension par somme cumulative

In [ ]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
indeces_to_remove = remove_cum_sum(data_preprocess.train, 0.95)
data_preprocess.train = np.delete(data_preprocess.train, indeces_to_remove, axis=1)
data_preprocess.test = np.delete(data_preprocess.test, indeces_to_remove, axis=1)
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    model_names = ['ComplementNB', 'XGBoost', 'SVC', 'SGD']
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_svc_sgd", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")